In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv("./project/train.csv")
data.set_index("encounter_id",inplace=True)

In [3]:
data.drop("country",axis=1,inplace=True)

In [4]:
data.drop("average_pulse_bpm",axis=1,inplace=True)

In [5]:
race_map={
    "Caucasian":"cauc",
    "AfricanAmerican":"afam",
    "Hispanic":"hisp",
    "Asian":"asian"}

In [6]:
payer_code_map={
    "?":"none",
    "MC":"mc",
    "HM":"hm",
    "SP":"sp",
    "BC":"bc",
    "MD":"md"}

In [7]:
admission_type_map={
    "Elective":"elec",
    "Emergency":"emerg",
    "Urgent":"emerg"}

In [8]:
medical_specialty_map={
    'Trauma': 'emerg',
    'Emergency/Trauma': 'emerg',
    'Family/GeneralPractice': 'fam',
    'GeneralPractice': 'fam',
    'Surgery-General': 'surg',
    'Surgeon': 'surg',
    'Surgery-Cardiovascular': 'surg',
    'Surgery-Cardiovascular/Thoracic': 'surg',
    'Surgery-Colon&Rectal': 'surg',
    'Surgery-Maxillofacial': 'surg',
    'Surgery-Neuro': 'surg',
    'Surgery-Pediatric': 'surg',
    'Surgery-PlasticwithinHeadandNeck': 'surg',
    'Surgery-Plastic': 'surg',
    'Surgery-Thoracic': 'surg',
    'Surgery-Vascular': 'surg',
    'SurgicalSpecialty': 'surg',
    'Orthopedics': 'ortho',
    'Orthopedics-Reconstructive': 'ortho',
    'InternalMedicine': 'int',
    'Cardiology': 'cardio',
    'Endocrinology': 'endo',
    'Endocrinology-Metabolism': 'endo',
    'Gastroenterology': 'gastro',
    'Hematology': 'hema',
    'Hematology/Oncology': 'hema',
    'InfectiousDiseases': 'infec',
    'Nephrology': 'neph',
    'Oncology': 'onco',
    'Pulmonology': 'pulm',
    'Rheumatology': 'rheum',
    'Urology': 'uro',
    'Neurology': 'neuro',
    'Neurophysiology': 'neuro',
    'Dermatology': 'derma',
    'Gynecology': 'obgy',
    'Obsterics&Gynecology-GynecologicOnco': 'obgy',
    'Obstetrics': 'obgy',
    'ObstetricsandGynecology': 'obgy',
    'Psychiatry': 'psychi',
    'Psychology': 'psycho',
    'Pediatrics-AllergyandImmunology': 'pedia',
    'Pediatrics-CriticalCare': 'pedia',
    'Pediatrics-EmergencyMedicine': 'pedia',
    'Pediatrics-Endocrinology': 'pedia',
    'Pediatrics-Hematology-Oncology': 'pedia',
    'Pediatrics-InfectiousDiseases': 'pedia',
    'Pediatrics-Neurology': 'pedia',
    'Pediatrics-Pulmonology': 'pedia',
    'Cardiology-Pediatric': 'pedia',
    'Psychiatry-Child/Adolescent': 'pedia',
    'Pediatrics':"pedia",
    'AllergyandImmunology':"imu",
    'Pathology':"path",
    'Proctology':"proc"}

In [9]:
discharge_disposition_map={
    "Discharged/transferred to SNF":"snf",
    "Discharged/transferred to a nursing facility certified under Medicaid but not certified under Medicare.":"snf",
    "Discharged/transferred to home with home health service":"home++",
    "Discharged/transferred to home under care of Home IV provider":"home+",
    "Discharged to home":"home",
    "Discharged/transferred to another rehab fac including rehab units of a hospital .":"rehab",
    "Discharged/transferred to another short term hospital":"short",
    "Discharged/transferred to another type of inpatient care institution":"long",
    "Discharged/transferred/referred to this institution for outpatient services":"out",
    "Discharged/transferred/referred another institution for outpatient services":"out",
    "Still patient or expected to return for outpatient services":"out",
    "Discharged/transferred to ICF":"icf",
    "Discharged/transferred to a federal health care facility.":"long",
    "Discharged/transferred to a long term care hospital.":"long",
    "Discharged/transferred within this institution to Medicare approved swing bed":"long",
    "Admitted as an inpatient to this hospital":"long",
    "Discharged/transferred/referred to a psychiatric hospital of psychiatric distinct part unit of a hospital":"long",
    "Left AMA":"left"}

In [10]:
admission_source_map={
    " Emergency Room":"emerg",
    "Transfer from a hospital":"transf",
    " Transfer from another health care facility":"transf",
    " Transfer from a Skilled Nursing Facility (SNF)":"transf",
    " Transfer from hospital inpt/same fac reslt in a sep claim":"transf",
    " Transfer from critial access hospital":"transf",
    " Transfer from Ambulatory Surgery Center":"transf",
    " Physician Referral":"ref",
    "Clinic Referral":"ref",
    "HMO Referral":"ref"}

In [11]:
glucose_test_result_map={"Norm":1,">200":2,">300":3}

In [12]:
a1c_test_result_map={"Norm":1,">7":2,">8":3}

In [13]:
change_in_meds_during_hospitalization_map={"Ch":1,"No":0}

In [14]:
prescribed_diabetes_meds_map={"Yes":1,"No":0}

In [15]:
mapped_categorical_features={
    "race":race_map,
    "payer_code":payer_code_map,
    "admission_type":admission_type_map,
    "medical_specialty":medical_specialty_map,
    "discharge_disposition":discharge_disposition_map,
    "admission_source":admission_source_map}
mapped_ordinal_features={
    "glucose_test_result":glucose_test_result_map,
    "a1c_test_result":a1c_test_result_map,
    "change_in_meds_during_hospitalization":change_in_meds_during_hospitalization_map,
    "prescribed_diabetes_meds":prescribed_diabetes_meds_map}

In [16]:
def safe(key,map:dict,default):
    return map.get(key,default)

In [17]:
gender_transformation=lambda gender: 1 if gender=="Female" else 0

In [18]:
weight_transformation=lambda weight: 0 if weight=="?" else 1

In [19]:
def age_transformation(age):
    return int(age[1])*10+5

In [20]:
def dampen_long_tail(value:int):
    return np.log(value+1)

In [21]:
long_tail_features=["emergency_visits_in_previous_year","outpatient_visits_in_previous_year",
    "inpatient_visits_in_previous_year"]

In [22]:
discrete_numerical_features=["length_of_stay_in_hospital","number_lab_tests",
    "non_lab_procedures","number_of_medications","number_diagnoses"]

In [23]:
def is_diabetes(code):
    if "." in code or code=="250":
        return 1
    else:
        return 0

In [24]:
def diabetes_kind(code):
    if "." in code:
        if (code.split(".")[1]).isalnum():
            return code.split(".")[1][0]
        else:
            return "unkown"
    if code=="250":
        return "unkown"
    return "none"

In [25]:
def diabetes_control(code):
    if "." in code:
        suffix=code.split(".")[1]
        if len(suffix)==2:
            if suffix[1] in {"0","1"}:
                return 1
            else:
                return 2
    return 0

In [26]:
def icd9_conversion(code):
    if code=="?": return "unkown"
    if code[0]=="M": return "m"
    if code[0]=="V": return "v"
    if code[0]=="E": return "e"
    if "." in code: return "b"
    try:
        code=int(code)
        if 1<=code<140: return "ipd"
        if 140<=code<240: return "neo"
        if 240<=code<280: return "enmi"
        if 280<=code<290: return "b"
        if 290<=code<320: return "men"
        if 320<=code<390: return "n"
        if 390<=code<460: return "c"
        if 460<=code<520: return "r"
        if 520<=code<580: return "d"
        if 580<=code<630: return "g"
        if 630<=code<680: return "pcp"
        if 680<=code<710: return "s"
        if 710<=code<740: return "mc"
        if 740<=code<760: return "cong"
        if 760<=code<780: return "pp"
        if 780<=code<800: return "ssi"
        if 800<=code<1000: return "ip"
    except ValueError:
        return "unkown"
    return "unkown"

In [27]:
def count_medication(med):
    count=0
    for char in med:
        if char=="'":
            count+=1
    return count//2

In [28]:
import re
def get_unique_medications(data):
    unique_medications=set()
    for meds in data["medication"]:
        for compound_med in (re.search(r'(?<=\[)(.*?)(?=\])',meds)[0]).split(sep=','):
            if compound_med!="":
                compound_med=compound_med.strip()
                for med in compound_med.split(sep='-'):
                    unique_medications.add(med.strip("'"))
    return unique_medications

In [29]:
unique_medications=get_unique_medications(data)

In [30]:
def preprocess(dataframe:pd.DataFrame):
    for feature,map in mapped_categorical_features.items():
        dataframe[feature]=dataframe[feature].map(lambda value: safe(value,map,"other")).astype("category")
    for feature,map in mapped_ordinal_features.items():
        dataframe[feature]=dataframe[feature].map(lambda value: safe(value,map,0)).astype("int64")
    dataframe["gender"]=dataframe["gender"].map(gender_transformation).astype("int64")
    dataframe["age"]=dataframe["age"].map(age_transformation,na_action="ignore").astype("float64")
    dataframe["weight"]=dataframe["weight"].map(weight_transformation).astype("int64")
    dataframe["total_number_of_visits_in_previous_year"]=0
    for feature in long_tail_features:
        dataframe["total_number_of_visits_in_previous_year"]+=dataframe[feature]
        #dataframe[feature]=dataframe[feature].map(dampen_long_tail).astype("float64")
    dataframe["total_number_of_visits_in_previous_year"]=\
        dataframe["total_number_of_visits_in_previous_year"].astype("int64")#.map(dampen_long_tail).astype("float64")
    dataframe["number_of_diabetes_meds"]=dataframe["medication"].map(count_medication).astype("int64")
    for feature in discrete_numerical_features:
        dataframe[feature]=dataframe[feature].astype("int64")
    for med in unique_medications:
        dataframe[med]=dataframe["medication"].map(lambda meds: med in meds).astype("int64")
    dataframe.drop("medication",axis=1,inplace=True)
    diagnoses=("primary_diagnosis","secondary_diagnosis","additional_diagnosis")
    for feature,diagnosis in zip(("diabetes_1","diabetes_2","diabetes_add"),diagnoses):
        dataframe[feature]=dataframe[diagnosis].map(is_diabetes).astype("int64")
    for feature,diagnosis in zip(("diabetes_1_kind",
        "diabetes_2_kind","diabetes_add_kind"),diagnoses):
        dataframe[feature]=dataframe[diagnosis].map(diabetes_kind).astype("category")
    for feature,diagnosis in zip(("diabetes_1_uncontrolled",
        "diabetes_2_uncontrolled","diabetes_add_uncontrolled"),diagnoses):
        dataframe[feature]=dataframe[diagnosis].map(diabetes_control).astype("int64")
    for diagnosis in diagnoses:
        dataframe[diagnosis]=dataframe[diagnosis].map(icd9_conversion).astype("category")
    return dataframe[sorted(dataframe.columns)].copy()

In [31]:
def rise_from_the_dead(dataframe:pd.DataFrame):
    disposition={"Expired",
        "Hospice / medical facility",
        "Hospice / home",
        "Expired at home. Medicaid only, hospice.",
        "Expired in a medical facility. Medicaid only, hospice."}
    return dataframe["discharge_disposition"].isin(disposition).values

In [32]:
def changed(observations:pd.Series):
    return int(len(observations.unique())!=1)

In [33]:
def count_max(observations:pd.Series):
    return sum(observations==observations.max())

In [34]:
def extract_full_history(training_data_frame:pd.DataFrame):
    historical_dataframe=pd.DataFrame(index=training_data_frame["patient_id"].unique())
    historical_dataframe.index.names=["patient_id"]
    aggregated_data_frame=training_data_frame.groupby("patient_id")
    historical_dataframe["observed_visits"]=(aggregated_data_frame.apply(lambda obs: len(obs)-1))
    historical_dataframe["changed_payer_code"]=aggregated_data_frame["payer_code"].apply(changed)
    translation={"elec":"admission_type_history_(elec)",
        "emerg":"admission_type_history_(emerg)",
        "other":"admission_type_history_(other)"}
    temporary_data_frame=aggregated_data_frame["admission_type"].value_counts().unstack().rename(translation)
    historical_dataframe.join(temporary_data_frame)
    historical_dataframe["changed_medical_specialty"]=aggregated_data_frame["medical_specialty"].apply(changed)
    translation={"emerg":"admission_source_history_(elec)",
        "transf":"admission_source_history_(transf)",
        "ref":"admission_source_history_(ref)",
        "other":"admission_source_history_(other)"}
    temporary_data_frame=aggregated_data_frame["admission_source"].value_counts().unstack().rename(translation)
    historical_dataframe.join(temporary_data_frame)
    for feature in (discrete_numerical_features+\
        ["number_of_diabetes_meds","total_number_of_visits_in_previous_year"]+\
        ["diabetes_1_uncontrolled","diabetes_2_uncontrolled","diabetes_add_uncontrolled"]+\
        list(mapped_ordinal_features)+long_tail_features):
        historical_dataframe["max_"+feature]=aggregated_data_frame[feature].max()
        historical_dataframe["number_of_max_"+feature]=aggregated_data_frame[feature].apply(count_max)
        historical_dataframe["mean_"+feature]=aggregated_data_frame[feature].mean()
    historical_dataframe["changed_primary_diagnosis"]=aggregated_data_frame["primary_diagnosis"].apply(changed)
    simple_sum_features=["diabetes_1","diabetes_2","diabetes_add"]+list(unique_medications)+\
        ["change_in_meds_during_hospitalization","prescribed_diabetes_meds"]
    temporary_data_frame=aggregated_data_frame[simple_sum_features].sum()
    historical_dataframe.join(temporary_data_frame)
    return historical_dataframe.copy()

In [35]:
def extract_simplified_history(training_data_frame:pd.DataFrame):
    historical_dataframe=pd.DataFrame(index=training_data_frame["patient_id"].unique())
    historical_dataframe.index.names=["patient_id"]
    aggregated_data_frame=training_data_frame.groupby("patient_id")
    historical_dataframe["observed_visits"]=aggregated_data_frame.apply(lambda obs: len(obs))
    return historical_dataframe

In [36]:
def fill_history_in(data_frame:pd.DataFrame,historical_data_frame:pd.DataFrame):
    data_frame=data_frame.join(historical_data_frame,how='left',on='patient_id')
    data_frame[data_frame[historical_data_frame.columns].isna()]=0
    data_frame.drop("patient_id",axis=1,inplace=True)
    return data_frame[sorted(data_frame.columns)].copy()

In [37]:
encoding={"features":["race","payer_code","admission_type","medical_specialty",
    "discharge_disposition","admission_source","primary_diagnosis","secondary_diagnosis",
    "additional_diagnosis","diabetes_1_kind","diabetes_2_kind","diabetes_add_kind"],
    "categories":[["cauc","afam","hisp","asian","other"],
        ["none","mc","hm","sp","bc","md","other"],
        ["elec","emerg","other"],
        ["emerg","fam","surg","ortho","int","cardio","endo","gastro","hema","infec","neph",
            "onco","pulm","rheum","uro","neuro","derma","obgy","psychi","psycho","pedia","imu",
            "path","proc","other"],
        ["snf","home","home+","home++","rehab","short","long","out","icf","left","other"],
        ["emerg","transf","ref","other"],
        ["unkown","m","v","e","b","ipd","neo","enmi","men","n","c","r","d","g","pcp","s",
        "mc","cong","pp","ssi","ip"],
        ["unkown","m","v","e","b","ipd","neo","enmi","men","n","c","r","d","g","pcp","s",
        "mc","cong","pp","ssi","ip"],
        ["unkown","m","v","e","b","ipd","neo","enmi","men","n","c","r","d","g","pcp","s",
        "mc","cong","pp","ssi","ip"],
        ["none","unkown"]+[str(i) for i in range(10)],
        ["none","unkown"]+[str(i) for i in range(10)],
        ["none","unkown"]+[str(i) for i in range(10)]]}

In [38]:
binary_target=(data["readmitted_binary"]=="Yes").astype("int64")
data.drop("readmitted_binary",axis=1,inplace=True)
data.drop("readmitted_multiclass",axis=1,inplace=True)
dead_index=np.arange(len(data))[rise_from_the_dead(data)]
data=preprocess(data)

In [39]:
from sklearn.preprocessing import OrdinalEncoder
encoder=OrdinalEncoder(categories=encoding["categories"],dtype=np.int64)
data[encoding["features"]]=encoder.fit_transform(data[encoding["features"]])

In [46]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
cross_validator=StratifiedKFold(n_splits=4,shuffle=True,random_state=0)
prediction=np.zeros(len(binary_target))
for fold,(train_index,val_index) in enumerate(cross_validator.split(data,binary_target)):
    train_index=train_index[~np.isin(train_index,dead_index)]
    val_index=val_index[~np.isin(val_index,dead_index)]
    X_train,X_val=data.iloc[train_index],data.iloc[val_index]
    y_train,y_val=binary_target.iloc[train_index],binary_target.iloc[val_index]
    train_history=extract_full_history(X_train)
    X_train=fill_history_in(X_train,train_history)
    X_val=fill_history_in(X_val,train_history)
    model=HistGradientBoostingClassifier(class_weight="balanced",learning_rate=0.1,
        random_state=0,max_iter=100,max_leaf_nodes=40)
    model.fit(X_train,y_train)
    prediction[val_index]=model.predict(X_val)
print("\nValidation:")
print(" Accuracy:  ",accuracy_score(binary_target,prediction).round(3))
print(" F1:  ",f1_score(binary_target,prediction).round(4))
print(" Confusion:\n",confusion_matrix(binary_target,prediction))


Validation:
 Accuracy:   0.818
 F1:   0.3603
 Confusion:
 [[54620  8666]
 [ 4299  3651]]


In [47]:
test_data=pd.read_csv("./project/test.csv")
test_data.set_index("encounter_id",inplace=True)
test_data.drop("country",axis=1,inplace=True)
test_data.drop("average_pulse_bpm",axis=1,inplace=True)

In [48]:
test_dead_index=np.arange(len(test_data))[rise_from_the_dead(test_data)]
test_data=preprocess(test_data)

In [49]:
test_data[encoding["features"]]=encoder.transform(test_data[encoding["features"]])

In [50]:
model=HistGradientBoostingClassifier(class_weight="balanced",learning_rate=0.1,
        random_state=0,max_iter=100,max_leaf_nodes=40)

In [51]:
training_mask=(~np.isin(np.arange(len(data)),dead_index))
training_data,training_target=data.iloc[training_mask],binary_target.iloc[training_mask]
training_history=extract_full_history(training_data)
training_data=fill_history_in(training_data,training_history)
model.fit(training_data,training_target)

HistGradientBoostingClassifier(class_weight='balanced', max_leaf_nodes=40,
                               random_state=0)

In [52]:
prediction=np.zeros(len(test_data))
test_mask=(~np.isin(np.arange(len(test_data)),test_dead_index))
temporary_data=test_data.iloc[test_mask]
temporary_data=fill_history_in(temporary_data,training_history)
prediction[test_mask]=model.predict(temporary_data)

In [53]:
prediction=pd.DataFrame(data={"readmitted_binary":prediction},
    index=test_data.index)
prediction["readmitted_binary"]=prediction["readmitted_binary"].replace({0:"No",1:"Yes"}).astype("object")
prediction.to_csv(path_or_buf="./Group31_Version03.csv",index_label="encounter_id")